# MLFlow Testing

This notebook tests MLFlow model management.

1. pip install mlflow

In [ ]:
import os
from pathlib import Path

from mlflow import log_metrics, log_params, log_artifact, set_experiment, start_run

from fantasy_py.inference import Model
from fantasy_py import log

_LOGGER = log.get_logger(__name__)


def archive_model(
    filepath: str,
    mlf_experiment=None,
    mlf_run_tags=None,
    mlf_run_name=None,
    mlf_run_description=None,
    model: Model | None = None,
):
    """
    archive the model at the filepath

    model - if None the load the model object from the filepath,
        if not None then assume this is the model for at the filepath
    """
    if model is None:
        model = Model.load(filepath)
    params_dict = {"name": model.name, "dt_trained": model.dt_trained}
    metrics = model.performance.copy()
    params_dict["metrics_season"] = metrics.pop("season")

    if mlf_experiment is not None:
        set_experiment(mlf_experiment)

    with start_run(
        run_name=mlf_run_name, tags=mlf_run_tags, description=mlf_run_description
    ):
        log_params(params_dict)
        log_metrics(metrics)

        artifacts_archived = [filepath]
        log_artifact(filepath)
        model_path_parent = Path(filepath).parent
        for artifact_path in model.artifacts:
            full_path = os.path.join(model_path_parent, artifact_path)
            artifacts_archived.append(full_path)
            log_artifact(full_path)

        _LOGGER.info(
            "Archived to MLFlow artifacts at: %s params: %s metrics: %s",
            artifacts_archived,
            params_dict,
            model.performance,
        )


def retrieve_model(name: str):
    raise NotImplementedError

In [ ]:
import os

from fantasy_py import get_git_desc


MLFLOW_EXPERIMENT = "Performance-Prediction 2023.08"
"""all performance predictions will exist under this experiement"""

MODEL_FILEPATH = os.path.join(
    "..", "2023.03", "LOL-player-DK.dk_performance_score.tpot.model"
)
MODEL_TAGS = {
    "sport": "LOL",
    "player-or-team": "player",
    "service": "draftkings",
    "target": "performance_score",
    "framework": "tpot",
    "fantasy-sha": get_git_desc(),
    "active": False,
    "cli-args": (
        "--seasons 2016 2017 2018 2019 2020 2021 2022 --no_teams "
        '--stats "*" --target_calc_stats "*performance_score" '
        "--hist_recent_games 3 --hist_recent_mode ma"
    ),
}
archive_model(
    MODEL_FILEPATH,
    mlf_experiment=MLFLOW_EXPERIMENT,
    mlf_run_name="test-name",
    mlf_run_description="test-description",
    mlf_run_tags=MODEL_TAGS,
)